In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dosonleung/jd_comment_with_label")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/jd_comment_with_label


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx
/kaggle/input/stopwords-hgd/stopwords.txt


In [3]:

from kagglehub import dataset_download
import pandas as pd
import os

# 下载数据集
path = dataset_download("dosonleung/jd_comment_with_label")
print(f"数据已下载到: {path}")

# 查找XLSX文件
xlsx_files = [f for f in os.listdir(path) if f.endswith('.xlsx')]

if xlsx_files:
    # 读取第一个XLSX文件
    file_path = os.path.join(path, xlsx_files[0])
    df = pd.read_excel(file_path)
    
    print(f"数据基本信息：")
    df.info()
    
    # 查看数据集行数和列数
    rows, columns = df.shape
    
    if rows < 5:
        # 样本行数少于5，则查看全量数据信息
        print(f"数据全部内容信息：")
        print(df.to_csv(sep='\t', na_rep='nan'))
    else:
        # 样本行数多于5，则查看数据前几行信息
        print(f"数据前几行内容信息：")
        print(df.head().to_csv(sep='\t', na_rep='nan'))
else:
    print("未找到XLSX文件，请检查数据集文件类型。")

数据已下载到: /kaggle/input/jd_comment_with_label
数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71818 entries, 0 to 71817
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   爬取时间(__time)           71818 non-null  object
 1   爬取链接(__url)            71818 non-null  object
 2   商品ID(product_id)       71818 non-null  int64 
 3   评价时间(publish_time)     71818 non-null  int64 
 4   评分（总分5分）(score)        71818 non-null  int64 
 5   评价内容(content)          71818 non-null  object
 6   评价者(author_name)       71818 non-null  object
 7   评价者会员等级(author_level)  71818 non-null  object
 8   商品sku(product_sku)     68032 non-null  object
 9   评价标签(tags)             71809 non-null  object
dtypes: int64(3), object(7)
memory usage: 5.5+ MB
数据前几行内容信息：
	爬取时间(__time)	爬取链接(__url)	商品ID(product_id)	评价时间(publish_time)	评分（总分5分）(score)	评价内容(content)	评价者(author_name)	评价者会员等级(author_level)	商品sku(product_sku)	评价标签(tags)
0	201

In [4]:
import numpy as np 
filtered_df = df[df['评价内容(content)'] != '此用户未填写评价内容']
# filtered_df = filtered_df[filtered_df['评价内容(content)'].str.len() > 15]
# filtered_df = filtered_df[(filtered_df['评分（总分5分）(score)'] !=3) ]

filtered_df['评分类别'] = np.where(filtered_df['评分（总分5分）(score)'] >= 4, 1, 0)

filtered_df[:2]


/tmp/ipykernel_19/148126958.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['评分类别'] = np.where(filtered_df['评分（总分5分）(score)'] >= 4, 1, 0)


,爬取时间(__time),爬取链接(__url),商品ID(product_id),评价时间(publish_time),评分（总分5分）(score),评价内容(content),评价者(author_name),评价者会员等级(author_level),商品sku(product_sku),评价标签(tags),评分类别
15,2019-03-08 00:50:31,https://sclub.jd.com/comment/productPageCommen...,4722324,1550636768,1,一般般，一分钱一分货吧,阿***5,铜牌会员,单机版 小D黑色,[],0
18,2019-03-08 00:50:31,https://sclub.jd.com/comment/productPageCommen...,4722324,1550652465,4,商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。,G***8,PLUS会员,单机版 小D黑色,[],1


In [5]:
selected_df = filtered_df[['评价内容(content)', '评分类别']]
selected_df[:5]

,评价内容(content),评分类别
15,一般般，一分钱一分货吧,0
18,商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。,1
19,。。。,1
22,刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局,1
25,还好还好还好还好红红火火好很好好,1


In [6]:
import re
from jieba import cut
from tensorflow.keras.preprocessing.text import Tokenizer
stopword_file='/kaggle/input/stopwords-hgd/stopwords.txt'
# 1. 定义清洗函数
def clean_text(text):
    # 去除非法字符（保留中文、字母、数字、空格）
    text = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', text)  
    # 去除停用词（需提前加载停用词表）
    stopwords = set(open(stopword_file, 'r', encoding='utf-8').read().split())
    words = [word for word in cut(text) if word not in stopwords]
    return ' '.join(words)  

# 2. 清洗数据
selected_df['cleaned_comment'] = selected_df['评价内容(content)'].apply(clean_text)

selected_df=selected_df[selected_df['cleaned_comment']!='']
# clean_comments = [clean_text(comment['评价内容(content)']) for comment in selected_df]

# # 3. 使用 Tokenizer 向量化
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(clean_comments)
# sequences = tokenizer.texts_to_sequences(clean_comments)

2025-05-24 06:52:46.632952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748069566.832172      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748069566.889174      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.622 seconds.
Prefix dict has been built successfully.
/tmp/ipykernel_19/375982273.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [7]:
selected_df[:5]

,评价内容(content),评分类别,cleaned_comment
15,一般般，一分钱一分货吧,0,一般般 一分钱 一分货
18,商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。,1,商品质量 很 好 很 满意 配送 速度 快 配送 员 态度 非常 好
22,刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局,1,刘慧敏 提莫 摸摸 摸 休息 泽 TCL 退咯 一组 婆婆 破鼓 规土局
25,还好还好还好还好红红火火好很好好,1,还好 还好 还好 还好 红红火火 好 很 好好
29,好好好好好好好好，好好好好好很好,1,好好 好好 好好 好好 好好 好 好好 很 好


In [8]:
selected_df_1 = selected_df[['cleaned_comment', '评分类别']]


In [9]:
print(len(selected_df_1[selected_df_1['评分类别']==0]))
print(len(selected_df_1[selected_df_1['评分类别']==1]))

1718
42265


In [10]:
# 复制数据十份
bad_cmt=selected_df_1[selected_df_1['评分类别']==0]
good_cmt=selected_df_1[selected_df_1['评分类别']==1]
df_duplicated = pd.concat([bad_cmt] * 10, ignore_index=True)

# 打乱数据顺序 (差评数据复制十份，好评数据保留15000)
bad_df = df_duplicated.sample(frac=1, random_state=42).reset_index(drop=True)
good_df = good_cmt.iloc[:15000]
print(len(bad_df))
print(len(good_df))

17180
15000


In [11]:
##合并好评和差评的结果
import pandas as pd
import numpy as np

# 假设df1和df2是两个要合并的DataFrame
# df1 = pd.DataFrame(...)
# df2 = pd.DataFrame(...)

# 方法1：使用concat + sample（推荐）
df_combined = pd.concat([bad_df, good_df], ignore_index=True)  # 按行合并，重置索引
selected_df_1 = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)  # 打乱顺序并重置索引


print(f"合并后形状: {selected_df_1.shape}")
print("数据示例:")
print(selected_df_1.head())

合并后形状: (32180, 2)
数据示例:
                                     cleaned_comment  评分类别
0                                   装上 挺 方便 内存 大些 更好     1
1         音质 b 家 就算 万元 以下 听个 响 蓝牙 会 很 卡顿 b 家买 外观 b 格     0
2                       手机 还 说 送 东西 没送 售后服务 态度 不好 差评     0
3  做功 好 面料 柔软 尺码 准确 穿着 舒服 胖子 穿 不显 肚子 游泳 温泉 都 最 主要...     1
4  买错 mva   va 面板 不行 开 滤 蓝   还开 调光   光感调 80   累眼 ...     1


In [12]:
# 3划分训练集和验证集
from sklearn.model_selection import train_test_split
import sklearn.utils


# 1. 划分数据集并重置索引
X_train, X_val, y_train, y_val = train_test_split(selected_df_1['cleaned_comment'],selected_df_1['评分类别'], test_size=0.2,stratify=selected_df_1['评分类别'], random_state=42)


# # 重建训练集 DataFrame
train_df = pd.concat([X_train.reset_index(drop=True), 
                      y_train.reset_index(drop=True)], 
                     axis=1)
val_df = pd.concat([X_val.reset_index(drop=True), 
                      y_val.reset_index(drop=True)], 
                     axis=1)
# train_df = train_df.reset_index(drop=True)
# val_df = val_df.reset_index(drop=True)




In [13]:
from transformers import AutoConfig, AutoModelForSequenceClassification
import torch

# 计算类别权重（修正版）
n_samples = len(y_train)
n_class0 = (y_train == 0).sum()
n_class1 = (y_train == 1).sum()
class_weights = [n_samples / (n_class0 * 2), n_samples / (n_class1 * 2)]

# 加载配置并设置类别权重
config = AutoConfig.from_pretrained('bert-base-chinese')
config.num_labels = 2
config.weight = class_weights  # 设置权重

# 初始化模型
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-chinese',
    config=config
)

# 冻结 BERT 所有层的参数
for param in model.bert.parameters():
    param.requires_grad = False

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# 统计可训练参数数量  可训练参数: 1538/102269186
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"可训练参数: {trainable_params}/{total_params}")

可训练参数: 1538/102269186


In [15]:
##Tokenizer
"""
1、原始文本拆分为token
2、加载词典
3、分词后token转换tensor类型的token_index
"""

from transformers import AutoTokenizer
##加载词典创建分词器
tokenizer=AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [16]:
good_score=val_df[val_df['评分类别']==1]
print(good_score.head())
bad_score=val_df[val_df['评分类别']==0]
print(bad_score.head())

        cleaned_comment  评分类别
4      质量 特别 好 朋友 都 说 买     1
5         没有 填写内容 默认 好评     1
6      好评 装进 笔记本 里 电脑 活     1
7         还 没用 不会 没弄 明白     1
10  多年 买 品牌 商卖 三文鱼 很 不错     1
                                     cleaned_comment  评分类别
0                       包装 坏   急用   只能 粘 一下 先   不 满意     0
1  杯子 收到 很久 才 评价 真的 很 好 物流 超快 以为 定制 会慢 一点 不到 三天 杯...     0
2           信号 差到 批爆 第一次 三星 还 信号 差到 批爆 大城市 信号 都 差到 爆     0
3                                  质量 太差 两米 不到 不值 这价     0
8            电 特别 不经用 充满 听歌 一个半 小时 左右 刚买 天 昨天 不 进去 电     0


In [17]:
# 假设y_train是Pandas Series
print("类别0的样本数:", (y_train == 0).sum())  # 统计值为0的样本数
print("类别1的样本数:", (y_train == 1).sum())  # 统计值为1的样本数

# 或者直接使用value_counts()查看完整分布
print(y_train.value_counts())

类别0的样本数: 13744
类别1的样本数: 12000
评分类别
0    13744
1    12000
Name: count, dtype: int64


In [18]:
from torch.utils.data import Dataset  

class CommentDataset(Dataset):
    def __init__(self, df, max_length=128):
        self.df = df.reset_index(drop=True)  # 确保索引连续
        self.max_length = max_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]  # 使用 iloc 按位置访问
        text = row['cleaned_comment']
        label = row['评分类别']
        
       
        
        return text,label

In [19]:
train_ds=CommentDataset(train_df)
val_ds=CommentDataset(val_df)

In [20]:
from torch.utils.data import DataLoader

def build_collate(tokenizer):
    def collate_fn(batch):
        ##文本分类语料输入语句类别标签
        
        sentents,labels=zip(*batch)

        ##tokenizer转换
        model_inputs=tokenizer(sentents,return_tensors='pt',padding='max_length',truncation=True)
        labels=torch.tensor(labels)
        return model_inputs,labels
    return collate_fn


##Dataloader
dl=DataLoader(train_ds,batch_size=16,shuffle=True,collate_fn=build_collate(tokenizer))
val_dl=DataLoader(val_ds,batch_size=16,shuffle=True,collate_fn=build_collate(tokenizer))


In [21]:
import torch
from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter

# 创建 TensorBoard 记录器
writer = SummaryWriter(log_dir='./runs/bert_experiment_fre')  # 日志保存路径

In [22]:
from transformers import AutoConfig, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import os


def train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, criterion, 
                device, epochs, log_dir="runs", patience=3):
    """训练模型并记录到TensorBoard"""
    # 初始化SummaryWriter
    writer = SummaryWriter(log_dir=log_dir)
    
    best_val_f1 = 0.0
    best_model_weights = None
    early_stopping_counter = 0
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 50)
        
        # 训练阶段
        model.train()
        total_train_loss = 0
        all_train_labels = []
        all_train_preds = []
        
        progress_bar = tqdm(train_dataloader, desc="Training")
        for batch_idx, batch in enumerate(progress_bar):
            # 数据移至设备
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 前向传播
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            
            # 梯度裁剪和优化
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            total_train_loss += loss.item()
            
            # 记录预测结果
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            all_train_preds.extend(preds)
            all_train_labels.extend(labels.cpu().numpy())
            
            # 更新进度条和TensorBoard
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
            writer.add_scalar('Train/Loss', loss.item(), epoch * len(train_dataloader) + batch_idx)
        
        # 计算训练指标
        avg_train_loss = total_train_loss / len(train_dataloader)
        train_accuracy = accuracy_score(all_train_labels, all_train_preds)
        train_f1 = f1_score(all_train_labels, all_train_preds, average='weighted')
        
        # 记录到TensorBoard
        writer.add_scalar('Train/Accuracy', train_accuracy, epoch)
        writer.add_scalar('Train/F1', train_f1, epoch)
        writer.add_scalar('Train/Avg_Loss', avg_train_loss, epoch)
        
        print(f"Training Loss: {avg_train_loss:.4f}")
        print(f"Training Accuracy: {train_accuracy:.4f}")
        print(f"Training F1 Score: {train_f1:.4f}")
        
        # 验证阶段
        val_loss, val_accuracy, val_f1, val_report = evaluate_model(model, val_dataloader, criterion, device)
        
        # 记录验证指标到TensorBoard
        writer.add_scalar('Validation/Loss', val_loss, epoch)
        writer.add_scalar('Validation/Accuracy', val_accuracy, epoch)
        writer.add_scalar('Validation/F1', val_f1, epoch)
        
        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        print(f"Validation F1 Score: {val_f1:.4f}")
        print("\nValidation Classification Report:")
        print(val_report)
        
        # 记录学习率
        current_lr = optimizer.param_groups[0]['lr']
        writer.add_scalar('Learning Rate', current_lr, epoch)
        
        # 保存最佳模型
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_weights = model.state_dict().copy()
            early_stopping_counter = 0
            model.save_pretrained('best_bert_model_dir')  # 保存模型权重和配置
            print("*** New best model saved! ***")
        else:
            early_stopping_counter += 1
            print(f"Early stopping counter: {early_stopping_counter}/{patience}")
            if early_stopping_counter >= patience:
                print("Early stopping triggered!")
                break
    
    # 关闭SummaryWriter
    writer.close()
    
    # 加载最佳模型
    if best_model_weights is not None:
        model.load_state_dict(best_model_weights)
        print(f"Loaded best model with validation F1: {best_val_f1:.4f}")
    
    return model, best_val_f1

def evaluate_model(model, dataloader, criterion, device):
    """评估模型并返回详细指标"""
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    progress_bar = tqdm(dataloader, desc="Evaluating")

    with torch.no_grad():
        for batch_idx, batch in enumerate(progress_bar):
            # 数据移至设备
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
        # for inputs,labels in dataloader:
        #     inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        #     labels = batch['labels'].to(device)
        
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            total_loss += loss.item()
            
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    report = classification_report(all_labels, all_preds, digits=4)
    
    return avg_loss, accuracy, f1, report

# 主函数示例
def main():
    # 假设已准备好数据和分词器
    # tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
    
    # 计算类别权重
    n_class0 = (y_train == 0).sum()
    n_class1 = (y_train == 1).sum()
    n_samples = len(y_train)
    class_weights = torch.tensor(
        [n_samples / (n_class0 * 2), n_samples / (n_class1 * 2)], 
        dtype=torch.float
    )
    
    # # 初始化模型
    # config = AutoConfig.from_pretrained('bert-base-chinese', num_labels=2)
    # model = AutoModelForSequenceClassification.from_pretrained(
    #     'bert-base-chinese', config=config
    # )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # 准备数据加载器
    # train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    # val_dataset = TextDataset(val_texts, val_labels, tokenizer)
    # train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    # val_dataloader = DataLoader(val_dataset, batch_size=16)
    
    # 优化器和调度器
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    total_steps = len(dl) * 3  # 3个训练轮次
    warmup_steps = int(total_steps * 0.1)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
    )
    
    # 损失函数
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
    
    # 开始训练
    trained_model, best_f1 = train_model(
        model=model,
        train_dataloader=dl,
        val_dataloader=val_dl,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=criterion,
        device=device,
        epochs=5,
        patience=2
    )
    
    return trained_model, best_f1



In [23]:
main()


Epoch 1/5
--------------------------------------------------


Training: 100%|██████████| 1609/1609 [07:07<00:00,  3.76it/s, loss=0.6597]


Training Loss: 0.6615
Training Accuracy: 0.6016
Training F1 Score: 0.6019


Evaluating: 100%|██████████| 403/403 [01:41<00:00,  3.96it/s]


Validation Loss: 0.5848
Validation Accuracy: 0.7282
Validation F1 Score: 0.7286

Validation Classification Report:
              precision    recall  f1-score   support

           0     0.7617    0.7145    0.7373      3436
           1     0.6947    0.7440    0.7185      3000

    accuracy                         0.7282      6436
   macro avg     0.7282    0.7292    0.7279      6436
weighted avg     0.7305    0.7282    0.7286      6436

*** New best model saved! ***

Epoch 2/5
--------------------------------------------------


...

Training: 100%|██████████| 1609/1609 [07:08<00:00,  3.75it/s, loss=0.4389]


Training Loss: 0.5362
Training Accuracy: 0.7634
Training F1 Score: 0.7633


Evaluating: 100%|██████████| 403/403 [01:42<00:00,  3.95it/s]

Validation Loss: 0.5160
Validation Accuracy: 0.7907
Validation F1 Score: 0.7909

Validation Classification Report:
              precision    recall  f1-score   support

           0     0.8128    0.7899    0.8012      3436
           1     0.7669    0.7917    0.7791      3000

    accuracy                         0.7907      6436
   macro avg     0.7898    0.7908    0.7901      6436
weighted avg     0.7914    0.7907    0.7909      6436

Early stopping counter: 2/2
Early stopping triggered!
Loaded best model with validation F1: 0.7909


(BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(21128, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-11): 12 x BertLayer(
           (attention): BertAttention(
             (self): BertSdpaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): La

In [24]:
val_df[:5]


,cleaned_comment,评分类别
0,包装 坏 急用 只能 粘 一下 先 不 满意,0
1,杯子 收到 很久 才 评价 真的 很 好 物流 超快 以为 定制 会慢 一点 不到 三天 杯...,0
2,信号 差到 批爆 第一次 三星 还 信号 差到 批爆 大城市 信号 都 差到 爆,0
3,质量 太差 两米 不到 不值 这价,0
4,质量 特别 好 朋友 都 说 买,1


In [25]:
val_df.to_csv('validation_data_fre--1.csv', index=False) 

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 加载模型和分词器
# model_path = "path/to/your/fine-tuned-model"  # 替换为你的模型路径
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 设置设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # 设置为评估模式
# 
# 待预测的文本
i=0
num=0
for index,raw in val_df[val_df['评分类别']==0].iterrows():
   text=raw['cleaned_comment']
   i=i+1
   if i==50:
       break

 
   inputs = tokenizer(text, return_tensors="pt").to(device)

    # 模型推理（关闭梯度计算以加速）
   with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取预测结果
   logits = outputs.logits
   predicted_class = torch.argmax(logits, dim=1).item()
    
    # 打印预测结果
   # print(f"预测类别索引: {predicted_class},实际类别{raw['评分类别']}")
   if(predicted_class==raw['评分类别']):
       num=num+1
print(num)

35


In [27]:
j=0
num1=0
for index,raw in val_df[val_df['评分类别']==1].iterrows():
   text=raw['cleaned_comment']
   j=j+1
   if j==50:
       break

 
   inputs = tokenizer(text, return_tensors="pt").to(device)

    # 模型推理（关闭梯度计算以加速）
   with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取预测结果
   logits = outputs.logits
   predicted_class = torch.argmax(logits, dim=1).item()
    
    # 打印预测结果
   print(text)
   print(f"预测类别索引: {predicted_class},实际类别{raw['评分类别']}")
   if predicted_class==raw['评分类别']:
       num1+=1
print(num1)

质量 特别 好 朋友 都 说 买
预测类别索引: 1,实际类别1
没有 填写内容 默认 好评
预测类别索引: 1,实际类别1
好评 装进 笔记本 里 电脑 活
预测类别索引: 1,实际类别1
还 没用 不会 没弄 明白
预测类别索引: 0,实际类别1
多年 买 品牌 商卖 三文鱼 很 不错
预测类别索引: 1,实际类别1
代 朋友 买 很 好 不错
预测类别索引: 1,实际类别1
物流 给力
预测类别索引: 1,实际类别1
东西 很 好 买 很 多次 都 不用 去 超市
预测类别索引: 1,实际类别1
收到 包装 很 好 没有 损坏 很漂亮 带 音乐 密码 还 验钞 小家伙 高兴 极 都 准备 存钱
预测类别索引: 1,实际类别1
宝贝 很漂亮 想要 感觉
预测类别索引: 1,实际类别1
做工 好 发货 快 非常 不错 卖家
预测类别索引: 1,实际类别1
赠品 还 不错 清晰度
预测类别索引: 1,实际类别1
快递 慢点 无所谓 说 功能
预测类别索引: 0,实际类别1
都 很 好 盒子 稍微 有点 尴尬 处理 里面 海绵 不 出去 每次 装 都 小心 摆放 好久
预测类别索引: 1,实际类别1
还 没 使用 过后 再 评价
预测类别索引: 1,实际类别1
很 好 手感 面料 不错 图案 好看
预测类别索引: 1,实际类别1
克服 电池电量 不够 问题 特意 选 原装 充电电池 充电 盒 希望 旅途 不会 有所 遗憾
预测类别索引: 1,实际类别1
宝宝 很 喜欢
预测类别索引: 1,实际类别1
包装 完好 物流 很快 价格 实惠 里面 东西
预测类别索引: 1,实际类别1
产品 收到 孩子 很 喜欢 质量 很 好 商家 服务态度 很 好 物流 很快
预测类别索引: 1,实际类别1
东西 非常 好 快递 很 力
预测类别索引: 1,实际类别1
不 知道 安装 打电话 戴尔 客服 客服 很 耐心 回答 值得 点赞
预测类别索引: 1,实际类别1
不错 小巧玲珑 音质 很 悦耳 赞
预测类别索引: 1,实际类别1
送 弟弟 还行 几天 想 购买
预测类别索引: 1,实际类别1
小 主板 机箱 好看 件 都 不是 大牌 看 耐用 不了
预测类别索引: 0,实际类别1
一如既往 咁 支持 台 电 固态 硬盘 系列
预测类别索引: 1,实际类别1
超级 棒 超级 棒 很 可

In [28]:
def predict_with_model(model, texts, tokenizer, device, label_map=None):
    """
    使用训练好的模型进行预测
    
    参数:
    - model: 已训练的分类模型
    - texts: 待预测的文本列表
    - tokenizer: 分词器
    - device: 计算设备
    - label_map: 类别索引到名称的映射字典
    
    返回:
    - 包含预测类别、概率的列表
    """
    model.eval()
    results = []
    
    # 处理单条文本的情况
    if isinstance(texts, str):
        texts = [texts]
    
    with torch.no_grad():
        for text in texts:
            # 编码文本
            inputs = tokenizer(text, return_tensors="pt").to(device)
            
            # 模型预测
            outputs = model(**inputs)
            logits = outputs.logits
            
            # 计算概率分布
            probs = torch.nn.functional.softmax(logits, dim=1)
            confidence, predicted_class_id = torch.max(probs, dim=1)
            
            # 转换为numpy
            predicted_class_id = predicted_class_id.item()
            confidence = confidence.item()
            
            # 映射类别名称（如果提供了映射）
            predicted_label = label_map[predicted_class_id] if label_map else predicted_class_id
            
            results.append({
                'text': text,
                'predicted_class_id': predicted_class_id,
                'predicted_label': predicted_label,
                'confidence': confidence,
                'probabilities': probs.cpu().numpy()[0].tolist()
            })
    
    return results

# 使用示例
label_map = {0: "负面", 1: "正面"}  # 根据实际情况修改
cmt_test=['商家发出的根本就不是一款，跟链接里的不一样，买的支架款也没有支架，真服了，发货巨慢，售后开个发票也磨磨叽叽到现在也没有拿到',     
          '保温效果一般，两个小时不到就凉了',
          '包装有脏污，盒子封口贴全是灰尘，送人用的，没办法只能把封口贴撕下来'
          ,'包裹运输中破损，持续20多天都没有退款',
          '纯纯塑料杯子，这杂能用，一股塑料的味道，劣质塑料',
          '有味道，不适合给孩子用，也不适合放热水，就是好看而已，真的一分钱一分货',
          '收到 货 之后 光顾 查询 序列号 去 没 仔细检查 外观 激活 之后 贴膜 才 发现 背 logo 上面 一点 刮伤 很 郁闷 很想 知道 刮痕',
          '风扇 噪音 大，没 想象 的 那么好',
          '这个保温杯真的超实用，670ml大容量满足全天饮水需求，316不锈钢材质安全无异味，保温效果长达12小时以上，粉色外观清新时尚。',
          '设计很漂亮，尺寸大小合适，包装仔细完整，手感很不错',
          '安装简单，价格实惠，宝贝确实不错，风力蛮大',
          '颜值高，白色很耐看，最喜欢的是米家的app远程控制，虽然需要有小爱音箱才能连接米家app,但是太好用了'
          ,'风量大声音小，可以调节风量，风速比较均匀，吹起来很舒服很自然',
         '期盼多久的镜子，太失望了',
          '包装有问题，把我手划破了',
          '除了好看一无是处',
          '太小了，装不了什么',]
model.to(device)
predictions = predict_with_model(model, cmt_test , tokenizer, device, label_map)

# 打印详细预测结果
for pred in predictions:
    print(f"文本: {pred['text']}")
    print(f"预测类别: {pred['predicted_label']} (索引:{pred['predicted_class_id']})")
    print(f"置信度: {pred['confidence']:.4f}")
    print(f"概率分布: {pred['probabilities']}")
    print("-" * 50)

文本: 商家发出的根本就不是一款，跟链接里的不一样，买的支架款也没有支架，真服了，发货巨慢，售后开个发票也磨磨叽叽到现在也没有拿到
预测类别: 负面 (索引:0)
置信度: 0.5349
概率分布: [0.534888744354248, 0.46511128544807434]
--------------------------------------------------
文本: 保温效果一般，两个小时不到就凉了
预测类别: 负面 (索引:0)
置信度: 0.5068
概率分布: [0.5067866444587708, 0.49321335554122925]
--------------------------------------------------
文本: 包装有脏污，盒子封口贴全是灰尘，送人用的，没办法只能把封口贴撕下来
预测类别: 正面 (索引:1)
置信度: 0.5784
概率分布: [0.4216057062149048, 0.57839435338974]
--------------------------------------------------
文本: 包裹运输中破损，持续20多天都没有退款
预测类别: 负面 (索引:0)
置信度: 0.5429
概率分布: [0.5429058074951172, 0.4570941627025604]
--------------------------------------------------
文本: 纯纯塑料杯子，这杂能用，一股塑料的味道，劣质塑料
预测类别: 负面 (索引:0)
置信度: 0.6543
概率分布: [0.6542813777923584, 0.3457186818122864]
--------------------------------------------------
文本: 有味道，不适合给孩子用，也不适合放热水，就是好看而已，真的一分钱一分货
预测类别: 正面 (索引:1)
置信度: 0.5406
概率分布: [0.45937520265579224, 0.5406247973442078]
--------------------------------------------------
文本: 收到 货 之后 光顾 查询 序列号 去 没 

In [29]:
tokenizer.save_pretrained('./tokenizer')


('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')